Exercise 1

In [ ]:
## No, it is not a good idea to try scaling the input features. 

Exercise 2

In [ ]:
## Yes, decreasing the max_depth would be a way to try and fit over-fitting.

Exercise 3

In [ ]:
## g). All of the above

Exercise 4

In [ ]:
## d). a. and c.

Exercise 5

Attribute: Outlook
-3 values: sunny, overcast, rain = 9-yes; 5-no
    Entropy = -(9/14)log2(9/14) - (5/14)log2(5/14) = 0.94
Entropy - Sunny
2-yes; 3-no
    Entropy(Sunny) = -(2/5)log2(2/5) - (3/5)log2(3/5) = 0.971
Entropy - Overcast
4-yes; 0-no
    Entropy(Overcast) = -(4/4)log2(4/4) - (0/4)log2(0/4) = 0
Entropy - Rain
3-yes; 2-no
    Entropy(Rain) = -(3/5)log2(3/5) - (2/5)log2(2/5) = 0.971
    
0.94 - (5/14) * 0.971 - (4/14) * 0 - (5/14) * 0.971 = 0.2464

Attribute: Temperature
-3 values: Hot, Mild, Cool = 9-yes; 5-no
    Entropy = -(9/14)log2(9/14) - (5/14)log2(5/14) = 0.94
Entropy - Hot
2-yes; 2-no
    Entropy(Hot) = -(2/4)log2(2/4) - (2/4)log2(2/4) = 1.0
Entropy - Mild
4-yes; 2-no
    Entropy(Mild) = -(4/6)log2(4/6) - (2/6)log2(2/6) = 0.9183
Entropy - Cool
3-yes; 1-no
    Entropy(Cool) = -(3/4)log2(3/4) - (1/4)log2(1/4) = 0.8113
    
0.94 - (4/14) * 1.0 - (6/14) * 0.8113 - (4/14) * 0.9183 = 0.0289

Attribute: Wind
-2 values: True, False = 9-yes; 5-no
    Entropy = -(9/14)log2(9/14) - (5/14)log2(5/14) = 0.94
Entropy - True
3-yes; 3-no
    Entropy(True) = -(3/6)log2(3/6) - (3/6)log2(3/6) = 1.0
Entropy - False
6-yes; 2-no
    Entropy(False) = -(6/8)log2(6/8) - (2/8)log2(2/8) = 0.8113

0.94 - (6/14) * 1.0 - (8/14) * 0.8113 = 0.0478

Attribute: Humidity
-2 values: High, Normal = 9-yes; 5-no
    Entropy = -(9/14)log2(9/14) - (5/14)log2(5/14) = 0.94
Entropy - High
3-yes; 4-no
    Entropy(High) = -(3/7)log2(3/7) - (4/7)log2(4/7) = 0.9852
Entropy - Normal
6-yes; 1-no
    Entropy(Normal) = -(6/7)log2(6/7) - (1/7)log2(1/7) = 0.5916

0.94 - (7/14) * 0.9852 - (7/14) * 0.5916 = 0.1516

In [ ]:
## The highest information gain would be the Outlook attribute. With 0.2646.

Exercise 6

In [29]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier


## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [30]:
## Removing the observations with N/As
heart = heart.dropna()

In [37]:
## Defining the input and target variables
X = heart[['male','age', 'education','currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes','totChol', 'sysBP', 'diaBP','BMI','heartRate', 'glucose']]
Y = heart['TenYearCHD']

for i in range(0, 100):
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

    ## Building the random forest model
    RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

    ## Extracting the feature importances
    importances= pd.DataFrame({'Feature': X_train.columns, 'Importance': RF_md.feature_importances_}) 
    importances = importances.sort_values(by = 'Importance', ascending = False)

## Average feature importances
importances_avg = np.mean(importances)
importances_avg = pd.DataFrame({'Feature': X_train.columns, 'Importance': RF_md.feature_importances_})
importances_avg = importances_avg.sort_values(by = 'Importance', ascending = False)
importances_avg

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3332: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


,Feature,Importance
1,age,0.194505
10,sysBP,0.187530
14,glucose,0.118696
11,diaBP,0.111127
9,totChol,0.067863
12,BMI,0.067631
7,prevalentHyp,0.065511
13,heartRate,0.046037
0,male,0.036518
4,cigsPerDay,0.035098


In [38]:
## Top 5 variables as predictors
## Defining the input and target variables for Random Forest
X = heart[['age', 'sysBP', 'diaBP','totChol', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [39]:
## Random Forest Model 1

for i in range(0, 100):
    ## Building the Random Forest model
    RF_md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred1 = RF_md1.predict_proba(X_test)[:, 1]

    ## Changing likelihoods to labels
    RF_labels1 = np.where(RF_pred1 < 0.1, 0, 1)

    ## Computing the recall score
    RF_recall1 = recall_score(Y_test, RF_labels1)

## Computing the average recall score
RF_recall1_avg = np.mean(RF_recall1)
print('The average recall score for the first random forest model is', RF_recall1_avg)

The average recall score for the first random forest model is 0.8157894736842105


In [40]:
## Random Forest Model 2

for i in range(0, 100):
    ## Building the Random Forest model
    RF_md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred2 = RF_md2.predict_proba(X_test)[:, 1]

    ## Changing likelihoods to labels
    RF_labels2 = np.where(RF_pred2 < 0.1, 0, 1)

    ## Computing the recall score
    RF_recall2 = recall_score(Y_test, RF_labels2)
    
## Computing the average recall score
RF_recall2_avg = np.mean(RF_recall2)
print('The average recall score for the second random forest model is', RF_recall2_avg)

The average recall score for the second random forest model is 0.8157894736842105


In [41]:
## Random Forest Model 3

for i in range(0, 100):
    ## Building the Random Forest model
    RF_md3 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred3 = RF_md3.predict_proba(X_test)[:, 1]

    ## Changing likelihoods to labels
    RF_labels3 = np.where(RF_pred3 < 0.1, 0, 1)

    ## Computing the recall score
    RF_recall3 = recall_score(Y_test, RF_labels3)
    
## Computing the average recall score
RF_recall3_avg = np.mean(RF_recall3)
print('The average recall score for the third random forest model is', RF_recall3_avg)

The average recall score for the third random forest model is 0.8245614035087719


In [ ]:
## The best random forest model would be the third, using a maximum depth of 7. The average recall score would be 0.8245617035087719, which would be the best to predict the TenYearCHD variable.